In [3]:
import torch
import json
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("wiki_lingua", "vietnamese")
dataset

/home/minhvn/miniconda3/envs/llama-adapter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['url', 'article'],
        num_rows: 6616
    })
})

In [16]:
len(dataset['train'])

6616

In [19]:
dataset["train"]['article'][0]['document']

['Hoa quả và rau thường rẻ hơn khi vào mùa. Thường thì các cửa hàng thực phẩm sẽ giảm giá cho các mặt hàng nông sản đang vào mùa vì họ biết khách hàng luôn chờ đợi những loại nào vào thời gian nào trong năm. Hơn nữa rau quả đúng mùa bao giờ cũng ngon hơn.  Vào mùa thu, bạn nên chọn các sản phẩm như bí ngô, bí mùa đông, cải bắp và táo. Mùa đông đến, bạn hãy tìm bí mùa đông, cải bắp và các loại củ khác. Mùa xuân là mùa của các loại rau củ như củ cải, rau lá xanh, hành lá và măng tây. Mùa hè là dịp tốt để thưởng thức nhiều loại rau quả, đơn cử như dưa hấu, ngô và các loại quả mọng. Có lẽ bạn cũng nhận thấy là giá cả của rau củ quả thường rẻ nhất trong mùa hè. Ngoài ra, vì nông sản mùa hè rẻ hơn nên bạn có thể mua nhiều để đông lạnh hoặc tự đóng hộp. Có lẽ bạn thích hoa quả và rau củ tươi hơn, nhưng thường thì bạn có thể tiết kiệm hơn nếu chọn sản phẩm đông lạnh hoặc đóng hộp. Những sản phẩm này cũng có lợi cho sức khỏe tương đương như sản phẩm tươi, nhưng bạn cần chọn các loại không thêm 

In [25]:
data = [d.get("document")[0] for d in dataset["train"]['article']]

In [50]:
type(data[0])

str

In [48]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
print(tokenizer.is_fast)
print(tokenizer.vocab_size)
train_data = tokenizer.encode(data)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 83001e09-6417-4d6a-9509-e8791b60ebbe)')' thrown while requesting HEAD https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json


True
119547


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [57]:
tokenizer.model_max_length

512

In [66]:
input_ids = tokenizer(
        data,
        return_tensors="pt",
        padding="max_length",
        max_length=tokenizer.model_max_length,
        truncation=True,
    ).input_ids

In [67]:
len(input_ids[0])

512

In [44]:
train_data[:100]

[101,
 164,
 112,
 15475,
 25308,
 10432,
 11859,
 10138,
 16591,
 186,
 29173,
 14789,
 12072,
 11603,
 27637,
 119,
 51635,
 38259,
 18004,
 10792,
 43872,
 18031,
 11992,
 24274,
 16807,
 39140,
 24287,
 11257,
 10792,
 20047,
 18031,
 47575,
 16913,
 21080,
 11603,
 27637,
 17819,
 10876,
 21820,
 38470,
 18031,
 53575,
 18643,
 39930,
 299,
 53725,
 11934,
 17031,
 27258,
 11603,
 13199,
 20264,
 27258,
 10504,
 10558,
 119,
 145,
 66316,
 42259,
 11859,
 10138,
 25308,
 82540,
 27637,
 18323,
 29656,
 13284,
 10743,
 10263,
 14789,
 119,
 29561,
 27637,
 23886,
 117,
 43094,
 19114,
 33090,
 10792,
 16913,
 24274,
 12552,
 57696,
 10743,
 16218,
 117,
 57696,
 27637,
 23164,
 117,
 45105,
 170,
 46916,
 10432,
 30185,
 10133,
 119,
 110433,
 23164,
 12002]

In [70]:
tokenizer.decode(input_ids[0])

'[CLS] Hoa quả và rau thường rẻ hơn khi vào mùa. Thường thì các cửa hàng thực phẩm sẽ giảm giá cho các mặt hàng nông sản đang vào mùa vì họ biết khách hàng luôn chờ đợi những loại nào vào thời gian nào trong năm. Hơn nữa rau quả đúng mùa bao giờ cũng ngon hơn. Vào mùa thu, bạn nên chọn các sản phẩm như bí ngô, bí mùa đông, cải bắp và táo. Mùa đông đến, bạn hãy tìm bí mùa đông, cải bắp và các loại củ khác. Mùa xuân là mùa của các loại rau củ như củ cải, rau lá xanh, hành lá và măng tây. Mùa hè là dịp tốt để thưởng thức nhiều loại rau quả, đơn cử như dưa hấu, ngô và các loại quả mọng. Có lẽ bạn cũng nhận thấy là giá cả của rau củ quả thường rẻ nhất trong mùa hè. Ngoài ra, vì nông sản mùa hè rẻ hơn nên bạn có thể mua nhiều để đông lạnh hoặc tự đóng hộp. Có lẽ bạn thích hoa quả và rau củ tươi hơn, nhưng thường thì bạn có thể tiết kiệm hơn nếu chọn sản phẩm đông lạnh hoặc đóng hộp. Những sản phẩm này cũng có lợi cho sức khỏe tương đương như sản phẩm tươi, nhưng bạn cần chọn các loại không t

In [72]:
tokenizer2 = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
print(tokenizer2.is_fast)
print(tokenizer2.vocab_size)

True
32000


In [80]:
tokenizer2.pad_token = tokenizer.unk_token
input_ids2 = tokenizer2(
        data,
        return_tensors="pt",
        padding="max_length",
        max_length=tokenizer2.model_max_length,
        truncation=True,
    ).input_ids

OverflowError: int too big to convert

In [77]:
tokenizer2.decode(input_ids2[0])

'<s> Hoa quả và rau thường rẻ hơn khi vào mùa. Thường thì các cửa hàng thực phẩm sẽ giảm giá cho các mặt hàng nông sản đang vào mùa vì họ biết khách hàng luôn chờ đợi những loại nào vào thời gian nào trong năm. Hơn nữa rau quả đúng mùa bao giờ cũng ngon hơn.  Vào mùa thu, bạn nên chọn các sản phẩm như bí ngô, bí mùa đông, cải bắp và táo. Mùa đông đến, bạn hãy tìm bí mùa đông, cải bắp và các loại củ khác. Mùa xuân là mùa của các loại rau củ như củ cải, rau lá xanh, hành lá và măng tây. Mùa hè là dịp tốt để thưởng thức nhiều loại rau quả, đơn cử như dưa hấu, ngô và các loại quả mọng. Có lẽ bạn cũng nhận thấy là giá cả của rau củ quả thường rẻ nhất trong mùa hè. Ngoài ra, vì nông sản mùa hè rẻ hơn nên bạn có thể mua nhiều để đông lạnh hoặc tự đóng hộp'

In [82]:
len(input_ids2)

6616

In [81]:
tokenizer2.model_max_length

1000000000000000019884624838656

In [91]:
# Split train/test
import numpy as np
np.random.seed(0)
perm = np.random.permutation(len(input_ids2))
split = int(len(perm) * 0.90)
print(f"SPLIT IS {split}")
train_indices = perm[:split]
eval_indices = perm[split:]
train_indices[:10]


SPLIT IS 5954


array([4396,  641, 4601, 3046, 1132,  451, 1523, 3043, 4995, 5189])

In [95]:
train_data = [input_ids2[i] for i in train_indices]
eval_data = [input_ids2[i] for i in eval_indices]

In [96]:
tokenizer2.decode(train_data[0])

'<s> Bạn có thể áp dụng phương pháp này trên cả Internet Explorer và Firefox.  Nếu dùng Microsoft Edge, bạn cần mở Internet Explorer để tiến hành, vì trình duyệt Edge không hỗ trợ tính năng này. Bất kể trình duyệt mặc định của bạn là gì thì khi nhấp đúp vào shortcut, trang web thường sẽ được mở bằng trình duyệt mà bạn đã dùng để tạo lối tắt. Mở chính xác trang mà bạn muốn tạo shortcut. Bạn có thể tạo lối tắt cho bất kỳ trang web nào nhưng vẫn cần phải đăng nhập nếu website luôn yêu cầu. Bạn cần thấy được màn hình nền để thao tác dễ dàng hơn. Bạn sẽ thấy biểu tượng trang web hơi mờ di chuyển cùng con trỏ chuột. Shortcut cùng tên với tiêu đề trang web  sẽ xuất hiện trên màn hình. Lối tắt có thể mang biểu tượng của website (nếu có). Nếu bạn dùng Internet Explorer để tạo shortcut thì trang web này sẽ luôn được mở trong Internet Explorer. Nhưng nếu bạn tạo bằng Firefox thì website sẽ '

In [100]:
train_data[0]

tensor([    1,   350, 30540, 29876, 28810,   266, 31957,  3976, 29886,   270,
        31620,   865,  1374, 30416, 30556,   865,  1374, 29976, 29886,   302,
        30001, 29891,   534,  5512,   274, 30643,  4685, 21508,   325, 30001,
        14418, 29889, 29871,   405, 30717, 29884,   270, 30071,   865,  7783,
        21086, 29892,   289, 30540, 29876,   274, 30884, 29876,   286,   228,
          190,   162,  4685, 21508, 29871, 30128, 31957, 19538, 30717, 29876,
          298, 24267, 29882, 29892,   325, 30097,   534, 30097, 29876, 29882,
          868, 29891, 30529, 29873, 21086,   413, 29882, 30069,   865,   298,
          228,   190,   154,   534, 31645,   260,  3642, 29882,   302, 30035,
          865,   302, 30001, 29891, 29889,   350, 31145, 29873,   413, 31957,
          534, 30097, 29876, 29882,   868, 29891, 30529, 29873,   286,   228,
          189,   186, 29883, 29871, 30128, 30740, 29876, 29882,   274, 31556,
        29874,   289, 30540, 29876, 18916,   330, 30097,   266, 

In [104]:
tokenizer2.decode([350, 30540, 29876])

'Bạn'

In [105]:
tokenizer2.vocab_size

32000

In [126]:
ix = torch.randint(high=len(train_data) - 512, size=(16,))
x = torch.stack([train_data[i] for i in ix])  # Stack training examples in a batch

In [122]:
type(train_data[0:10])

list

In [127]:
ix

tensor([4505,  201, 1845, 2473,  658, 3264, 5126, 3473,  423, 4524, 3429, 2978,
        2391, 5340, 1932, 2379])